Lien de téléchargement des données: https://cvml.ista.ac.at/AwA2/        

13GB file : https://cvml.ista.ac.at/AwA2/AwA2-data.zip

In [1]:
import sys
import cv2 # Pour utiliser open_cv, il faut la version de python est 3.7
import os
import csv

import numpy as np 
import pandas as pd 
import math

import torch 
from torch.utils.data import Dataset, DataLoader
import torchvision 
from torchvision.io import read_image
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler


import matplotlib.pyplot as plt

np.random.seed(0)

In [2]:
# Constant. Should be the path to the folder named JPEGImages, containing the 33K images in its subfolders.
JPEGIMAGES_FOLDER_PATH = "C:/Users/1/Desktop/data_ift3710/Animals_with_Attributes2/JPEGImages/"
path_project = "C:/Users/1/Desktop/data_ift3710/Animals_with_Attributes2/"

In [3]:
labels_dirs = os.listdir(JPEGIMAGES_FOLDER_PATH)
print(labels_dirs)
len(labels_dirs) # 50 labels / subdirectories

['antelope', 'bat', 'beaver', 'blue+whale', 'bobcat', 'buffalo', 'chihuahua', 'chimpanzee', 'collie', 'cow', 'dalmatian', 'deer', 'dolphin', 'elephant', 'fox', 'german+shepherd', 'giant+panda', 'giraffe', 'gorilla', 'grizzly+bear', 'hamster', 'hippopotamus', 'horse', 'humpback+whale', 'killer+whale', 'leopard', 'lion', 'mole', 'moose', 'mouse', 'otter', 'ox', 'persian+cat', 'pig', 'polar+bear', 'rabbit', 'raccoon', 'rat', 'rhinoceros', 'seal', 'sheep', 'siamese+cat', 'skunk', 'spider+monkey', 'squirrel', 'tiger', 'walrus', 'weasel', 'wolf', 'zebra']


50

In [4]:
def get_num_label(label):
    return labels_dirs.index(labels_dirs)

# Note : Some labels have a low number of images. 

## Possible solutions to explore : 
    Data augmentation : creating new training data by applying random transformations to existing images, such as rotating, cropping, or flipping them.

In [5]:
def find_num_images_per_label(img_dir = JPEGIMAGES_FOLDER_PATH): 
    """ 
    USEFUL FOR SAMPLING.
    Return a dict with keys as the 50 labels, and values being the number of images in each subdirectory corresponding to label
    and a second dict with the relative numbers (proportion) for every label compared to the total number of images (useful for sampling)"""
    labels_dirs = os.listdir(img_dir)
    num_images_per_label = dict.fromkeys(labels_dirs)
    proportions_images_per_label = dict.fromkeys(labels_dirs)
    total_num_images = 0

    # Update absolute number of images per label
    for i, label in enumerate(labels_dirs) : 
        specific_label_path = os.path.join(img_dir, labels_dirs[i])
        num_images_label = len(os.listdir(specific_label_path))
        total_num_images += num_images_label
        num_images_per_label[label] = num_images_label

    # Update relative number of images per label (proportion)
    for i, label in enumerate(labels_dirs) : 
        num_images_label = num_images_per_label[label]
        proportion_label = round(num_images_label / total_num_images, 4)
        proportions_images_per_label[label] = proportion_label

    return num_images_per_label, proportions_images_per_label

num_images_per_label, proportions_images_per_label = find_num_images_per_label()

In [6]:
ANNOTATIONS_FILENAME = 'annotations.csv'

def create_annotations_csv_file(annotations_filename = ANNOTATIONS_FILENAME, img_dir = JPEGIMAGES_FOLDER_PATH) : 
    """ 
    Create a csv annotations_file, annotations.csv, with two columns, in the format : 
                        path/to/image, label
    
    The annotation csv is necessary for DataLoader.
    """

    labels_dirs:list = os.listdir(img_dir)
   
    if os.path.exists(annotations_filename):
        os.remove(annotations_filename)
        print(f'Deleted existent {ANNOTATIONS_FILENAME} file.\n ---------------------------')
    
    with open(annotations_filename, 'w', newline='') as file :
        writer = csv.writer(file, dialect='excel', delimiter=',')

        for i, label in enumerate(labels_dirs) : 

            specific_label_path = os.path.join(img_dir, label)
            images_names = os.listdir(specific_label_path)

            for j, image_name in enumerate(images_names):
                full_path_to_img= os.path.join(specific_label_path, image_name)
                full_path_to_img= os.path.join(label, image_name)

                row = [full_path_to_img, label]
                writer.writerow(row)

    print(f'Sucessfully created {ANNOTATIONS_FILENAME} file.')

#
create_annotations_csv_file()

Deleted existent annotations.csv file.
 ---------------------------
Sucessfully created annotations.csv file.


In [7]:
class AWA2Dataset(Dataset): # Dataset class to serve as input for the DataLoader.
    """ 
    Dataset class to serve as input for the DataLoader.
    Implements all the required methods and more. 
    """

    def __init__(self, annotations_file=ANNOTATIONS_FILENAME, img_dir=JPEGIMAGES_FOLDER_PATH, 
                transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

        numbers_infos_dicts: tuple[dict,dict] = find_num_images_per_label(img_dir=JPEGIMAGES_FOLDER_PATH)
        self.num_images_per_label = numbers_infos_dicts[0]
        self.proportions_images_per_label = numbers_infos_dicts[1]

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        # img_path = self.img_labels.iloc[idx, 0]
        label = self.img_labels.iloc[idx, 1]

        image = read_image(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [8]:
dataset = AWA2Dataset()

## TODO : Change transforms. Currently this is not useful.
dataset.transform = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.Resize((256,256)),
                    transforms.CenterCrop((224,224)),
                    transforms.Grayscale(num_output_channels=3),
                    transforms.ToTensor(), # Already a tensor as implemented in Dataset class with the reaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                ])

# Testing. All good
random_index = np.random.randint(0, len(dataset))
image, label = dataset[random_index]

In [9]:
# Experiment with DataLoader. Everything works good
dataloader = DataLoader(dataset = dataset, batch_size=4, shuffle=True)
dataiter = iter(dataloader)
data = next(dataiter)

images, labels = data 


###   AlexNet  ###

In [10]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

#Model description
AlexNet_model.eval()

Using cache found in C:\Users\1/.cache\torch\hub\pytorch_vision_v0.6.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

### CUDA ###

In [11]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [12]:
torch.cuda.is_available()

True

### Question: nombre de noeud ###

In [13]:
nb_noeud = 4096

In [14]:
AlexNet_model.classifier[4] = torch.nn.Linear(4096,nb_noeud)
AlexNet_model.classifier[6] = torch.nn.Linear(nb_noeud,50)

In [15]:
import sys
print(sys.version)

3.8.16 (default, Mar  2 2023, 03:18:16) [MSC v.1916 64 bit (AMD64)]


In [16]:
AlexNet_model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

### Question: check dimension de dataloader   ###

###  Split training data and test data ###

In [17]:
batch_size = 32

In [18]:
dataloader = DataLoader(dataset = dataset, batch_size=batch_size, shuffle=True)
train_size = int(0.8*len(dataset))
test_size = len(dataset) - train_size

In [19]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset,[train_size,test_size])

In [20]:
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False, num_workers=0)

###   transfomer labels  ###

In [21]:
path_class = path_project +"classes.txt"
class_animal = pd.read_table(path_class,header= None)

In [22]:
animals = class_animal[1]

In [23]:
dict_label_animal = {}

In [24]:
n = 0
for i in range(0,len(animals)):
    dict_label_animal[animals[i]] = n
    n+=1

In [25]:
def label_to_num(tuple_labels):
    list_labels =[]
    for tuple_label in tuple_labels:
        list_labels.append(dict_label_animal[tuple_label])
    return torch.tensor(list_labels)   

###   Loss function  ###

In [26]:
lr = 0.001

In [27]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(AlexNet_model.parameters(), lr= lr , momentum=0.9)

###  Training  ###

In [28]:
for epoch in range(30):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data[0].to(device)
        labels = label_to_num(data[1]).to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = AlexNet_model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss ))
        running_loss = 0.0

print('Finished Training of AlexNet')

[1,     1] loss: 4.019
[1,     2] loss: 3.981
[1,     3] loss: 3.984
[1,     4] loss: 3.960
[1,     5] loss: 3.972
[1,     6] loss: 3.981
[1,     7] loss: 3.704
[1,     8] loss: 3.892
[1,     9] loss: 3.518
[1,    10] loss: 3.130
[1,    11] loss: 3.552
[1,    12] loss: 3.714
[1,    13] loss: 3.575
[1,    14] loss: 3.406
[1,    15] loss: 3.043
[1,    16] loss: 3.444
[1,    17] loss: 3.080
[1,    18] loss: 3.211
[1,    19] loss: 3.104
[1,    20] loss: 2.859
[1,    21] loss: 2.777
[1,    22] loss: 3.082
[1,    23] loss: 3.189
[1,    24] loss: 3.022
[1,    25] loss: 2.689
[1,    26] loss: 2.594
[1,    27] loss: 2.959
[1,    28] loss: 2.685
[1,    29] loss: 1.964
[1,    30] loss: 2.581
[1,    31] loss: 2.170
[1,    32] loss: 2.501
[1,    33] loss: 2.422
[1,    34] loss: 2.181
[1,    35] loss: 2.562
[1,    36] loss: 1.932
[1,    37] loss: 1.917
[1,    38] loss: 2.224
[1,    39] loss: 2.163
[1,    40] loss: 1.815
[1,    41] loss: 2.036
[1,    42] loss: 2.087
[1,    43] loss: 2.292
[1,    44] 

[1,   358] loss: 1.224
[1,   359] loss: 0.851
[1,   360] loss: 1.284
[1,   361] loss: 1.341
[1,   362] loss: 1.618
[1,   363] loss: 1.177
[1,   364] loss: 1.037
[1,   365] loss: 1.163
[1,   366] loss: 1.007
[1,   367] loss: 0.898
[1,   368] loss: 1.410
[1,   369] loss: 1.322
[1,   370] loss: 0.903
[1,   371] loss: 1.407
[1,   372] loss: 1.413
[1,   373] loss: 1.220
[1,   374] loss: 1.053
[1,   375] loss: 1.142
[1,   376] loss: 1.018
[1,   377] loss: 0.875
[1,   378] loss: 0.769
[1,   379] loss: 0.946
[1,   380] loss: 1.335
[1,   381] loss: 0.866
[1,   382] loss: 0.602
[1,   383] loss: 0.972
[1,   384] loss: 1.207
[1,   385] loss: 1.043
[1,   386] loss: 1.549
[1,   387] loss: 0.847
[1,   388] loss: 1.237
[1,   389] loss: 1.132
[1,   390] loss: 1.121
[1,   391] loss: 1.310
[1,   392] loss: 0.627
[1,   393] loss: 1.199
[1,   394] loss: 0.692
[1,   395] loss: 0.710
[1,   396] loss: 0.581
[1,   397] loss: 0.677
[1,   398] loss: 0.888
[1,   399] loss: 1.014
[1,   400] loss: 1.015
[1,   401] 

[1,   715] loss: 1.032
[1,   716] loss: 0.620
[1,   717] loss: 0.899
[1,   718] loss: 0.983
[1,   719] loss: 1.074
[1,   720] loss: 1.118
[1,   721] loss: 1.188
[1,   722] loss: 0.883
[1,   723] loss: 0.842
[1,   724] loss: 1.190
[1,   725] loss: 0.749
[1,   726] loss: 1.173
[1,   727] loss: 0.732
[1,   728] loss: 1.194
[1,   729] loss: 0.945
[1,   730] loss: 0.900
[1,   731] loss: 0.780
[1,   732] loss: 0.573
[1,   733] loss: 1.353
[1,   734] loss: 1.015
[1,   735] loss: 0.920
[1,   736] loss: 1.260
[1,   737] loss: 1.294
[1,   738] loss: 0.412
[1,   739] loss: 1.044
[1,   740] loss: 0.639
[1,   741] loss: 0.849
[1,   742] loss: 0.711
[1,   743] loss: 0.472
[1,   744] loss: 1.353
[1,   745] loss: 0.789
[1,   746] loss: 1.089
[1,   747] loss: 1.025
[1,   748] loss: 0.737
[1,   749] loss: 0.659
[1,   750] loss: 0.721
[1,   751] loss: 0.616
[1,   752] loss: 1.195
[1,   753] loss: 0.501
[1,   754] loss: 0.745
[1,   755] loss: 1.241
[1,   756] loss: 0.907
[1,   757] loss: 1.294
[1,   758] 

[2,   139] loss: 0.669
[2,   140] loss: 0.393
[2,   141] loss: 0.750
[2,   142] loss: 0.879
[2,   143] loss: 0.914
[2,   144] loss: 0.642
[2,   145] loss: 0.632
[2,   146] loss: 0.589
[2,   147] loss: 1.064
[2,   148] loss: 0.577
[2,   149] loss: 0.669
[2,   150] loss: 0.349
[2,   151] loss: 0.907
[2,   152] loss: 0.521
[2,   153] loss: 0.718
[2,   154] loss: 1.169
[2,   155] loss: 0.613
[2,   156] loss: 0.902
[2,   157] loss: 0.653
[2,   158] loss: 0.443
[2,   159] loss: 0.934
[2,   160] loss: 0.441
[2,   161] loss: 0.830
[2,   162] loss: 0.640
[2,   163] loss: 0.474
[2,   164] loss: 0.508
[2,   165] loss: 0.541
[2,   166] loss: 0.489
[2,   167] loss: 0.401
[2,   168] loss: 0.623
[2,   169] loss: 0.929
[2,   170] loss: 0.576
[2,   171] loss: 0.785
[2,   172] loss: 0.786
[2,   173] loss: 0.805
[2,   174] loss: 0.906
[2,   175] loss: 0.785
[2,   176] loss: 0.622
[2,   177] loss: 0.354
[2,   178] loss: 0.470
[2,   179] loss: 1.196
[2,   180] loss: 0.447
[2,   181] loss: 0.675
[2,   182] 

[2,   496] loss: 0.592
[2,   497] loss: 0.446
[2,   498] loss: 0.358
[2,   499] loss: 0.654
[2,   500] loss: 0.661
[2,   501] loss: 0.736
[2,   502] loss: 0.479
[2,   503] loss: 0.767
[2,   504] loss: 0.430
[2,   505] loss: 0.881
[2,   506] loss: 0.756
[2,   507] loss: 0.507
[2,   508] loss: 1.052
[2,   509] loss: 0.580
[2,   510] loss: 0.569
[2,   511] loss: 0.616
[2,   512] loss: 0.617
[2,   513] loss: 0.689
[2,   514] loss: 0.497
[2,   515] loss: 0.374
[2,   516] loss: 0.879
[2,   517] loss: 0.330
[2,   518] loss: 0.463
[2,   519] loss: 0.564
[2,   520] loss: 0.481
[2,   521] loss: 0.526
[2,   522] loss: 0.517
[2,   523] loss: 0.534
[2,   524] loss: 0.717
[2,   525] loss: 0.592
[2,   526] loss: 0.831
[2,   527] loss: 0.649
[2,   528] loss: 0.669
[2,   529] loss: 0.182
[2,   530] loss: 0.517
[2,   531] loss: 0.500
[2,   532] loss: 0.635
[2,   533] loss: 0.376
[2,   534] loss: 0.312
[2,   535] loss: 0.849
[2,   536] loss: 0.418
[2,   537] loss: 0.442
[2,   538] loss: 0.389
[2,   539] 

...

[29,   222] loss: 0.001
[29,   223] loss: 0.002
[29,   224] loss: 0.001
[29,   225] loss: 0.000
[29,   226] loss: 0.000
[29,   227] loss: 0.001
[29,   228] loss: 0.000
[29,   229] loss: 0.000
[29,   230] loss: 0.114
[29,   231] loss: 0.000
[29,   232] loss: 0.001
[29,   233] loss: 0.004
[29,   234] loss: 0.001
[29,   235] loss: 0.016
[29,   236] loss: 0.000
[29,   237] loss: 0.000
[29,   238] loss: 0.000
[29,   239] loss: 0.003
[29,   240] loss: 0.000
[29,   241] loss: 0.001
[29,   242] loss: 0.001
[29,   243] loss: 0.001
[29,   244] loss: 0.000
[29,   245] loss: 0.091
[29,   246] loss: 0.001
[29,   247] loss: 0.001
[29,   248] loss: 0.000
[29,   249] loss: 0.001
[29,   250] loss: 0.004
[29,   251] loss: 0.001
[29,   252] loss: 0.000
[29,   253] loss: 0.000
[29,   254] loss: 0.001
[29,   255] loss: 0.001
[29,   256] loss: 0.001
[29,   257] loss: 0.001
[29,   258] loss: 0.001
[29,   259] loss: 0.001
[29,   260] loss: 0.000
[29,   261] loss: 0.000
[29,   262] loss: 0.013
[29,   263] loss

[29,   564] loss: 0.000
[29,   565] loss: 0.001
[29,   566] loss: 0.005
[29,   567] loss: 0.000
[29,   568] loss: 0.000
[29,   569] loss: 0.000
[29,   570] loss: 0.000
[29,   571] loss: 0.001
[29,   572] loss: 0.001
[29,   573] loss: 0.000
[29,   574] loss: 0.000
[29,   575] loss: 0.000
[29,   576] loss: 0.000
[29,   577] loss: 0.000
[29,   578] loss: 0.000
[29,   579] loss: 0.000
[29,   580] loss: 0.000
[29,   581] loss: 0.000
[29,   582] loss: 0.000
[29,   583] loss: 0.000
[29,   584] loss: 0.001
[29,   585] loss: 0.108
[29,   586] loss: 0.000
[29,   587] loss: 0.000
[29,   588] loss: 0.000
[29,   589] loss: 0.000
[29,   590] loss: 0.000
[29,   591] loss: 0.000
[29,   592] loss: 0.000
[29,   593] loss: 0.000
[29,   594] loss: 0.000
[29,   595] loss: 0.000
[29,   596] loss: 0.000
[29,   597] loss: 0.000
[29,   598] loss: 0.148
[29,   599] loss: 0.000
[29,   600] loss: 0.000
[29,   601] loss: 0.000
[29,   602] loss: 0.001
[29,   603] loss: 0.000
[29,   604] loss: 0.000
[29,   605] loss

[29,   906] loss: 0.106
[29,   907] loss: 0.000
[29,   908] loss: 0.116
[29,   909] loss: 0.001
[29,   910] loss: 0.001
[29,   911] loss: 0.002
[29,   912] loss: 0.000
[29,   913] loss: 0.001
[29,   914] loss: 0.001
[29,   915] loss: 0.000
[29,   916] loss: 0.000
[29,   917] loss: 0.001
[29,   918] loss: 0.006
[29,   919] loss: 0.001
[29,   920] loss: 0.001
[29,   921] loss: 0.002
[29,   922] loss: 0.003
[29,   923] loss: 0.002
[29,   924] loss: 0.002
[29,   925] loss: 0.004
[29,   926] loss: 0.000
[29,   927] loss: 0.002
[29,   928] loss: 0.001
[29,   929] loss: 0.002
[29,   930] loss: 0.001
[29,   931] loss: 0.000
[29,   932] loss: 0.001
[29,   933] loss: 0.001
[30,     1] loss: 0.005
[30,     2] loss: 0.001
[30,     3] loss: 0.003
[30,     4] loss: 0.001
[30,     5] loss: 0.001
[30,     6] loss: 0.001
[30,     7] loss: 0.001
[30,     8] loss: 0.000
[30,     9] loss: 0.001
[30,    10] loss: 0.001
[30,    11] loss: 0.000
[30,    12] loss: 0.000
[30,    13] loss: 0.000
[30,    14] loss

[30,   315] loss: 0.001
[30,   316] loss: 0.001
[30,   317] loss: 0.003
[30,   318] loss: 0.001
[30,   319] loss: 0.000
[30,   320] loss: 0.000
[30,   321] loss: 0.001
[30,   322] loss: 0.002
[30,   323] loss: 0.000
[30,   324] loss: 0.000
[30,   325] loss: 0.001
[30,   326] loss: 0.001
[30,   327] loss: 0.001
[30,   328] loss: 0.001
[30,   329] loss: 0.001
[30,   330] loss: 0.000
[30,   331] loss: 0.000
[30,   332] loss: 0.000
[30,   333] loss: 0.002
[30,   334] loss: 0.000
[30,   335] loss: 0.000
[30,   336] loss: 0.001
[30,   337] loss: 0.001
[30,   338] loss: 0.001
[30,   339] loss: 0.000
[30,   340] loss: 0.000
[30,   341] loss: 0.000
[30,   342] loss: 0.000
[30,   343] loss: 0.001
[30,   344] loss: 0.000
[30,   345] loss: 0.000
[30,   346] loss: 0.000
[30,   347] loss: 0.000
[30,   348] loss: 0.002
[30,   349] loss: 0.001
[30,   350] loss: 0.001
[30,   351] loss: 0.001
[30,   352] loss: 0.002
[30,   353] loss: 0.002
[30,   354] loss: 0.001
[30,   355] loss: 0.001
[30,   356] loss

[30,   657] loss: 0.000
[30,   658] loss: 0.001
[30,   659] loss: 0.002
[30,   660] loss: 0.002
[30,   661] loss: 0.001
[30,   662] loss: 0.001
[30,   663] loss: 0.001
[30,   664] loss: 0.000
[30,   665] loss: 0.001
[30,   666] loss: 0.001
[30,   667] loss: 0.000
[30,   668] loss: 0.002
[30,   669] loss: 0.001
[30,   670] loss: 0.002
[30,   671] loss: 0.000
[30,   672] loss: 0.001
[30,   673] loss: 0.000
[30,   674] loss: 0.001
[30,   675] loss: 0.001
[30,   676] loss: 0.001
[30,   677] loss: 0.001
[30,   678] loss: 0.201
[30,   679] loss: 0.001
[30,   680] loss: 0.003
[30,   681] loss: 0.001
[30,   682] loss: 0.002
[30,   683] loss: 0.002
[30,   684] loss: 0.000
[30,   685] loss: 0.001
[30,   686] loss: 0.001
[30,   687] loss: 0.002
[30,   688] loss: 0.000
[30,   689] loss: 0.001
[30,   690] loss: 0.001
[30,   691] loss: 0.001
[30,   692] loss: 0.002
[30,   693] loss: 0.002
[30,   694] loss: 0.001
[30,   695] loss: 0.001
[30,   696] loss: 0.001
[30,   697] loss: 0.001
[30,   698] loss

### Test ###

In [29]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images = data[0].to(device)
        labels = label_to_num(data[1]).to(device)
        outputs = AlexNet_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 78 %
